In [1]:
#The AAPC website has an error on the Anesthesia page where it shows the wrong description of the parent
import bs4 as bs
import requests
import pandas as pd

In [2]:
#pretend you're a real person and not a robot
headers={'User-agent' : 'Mozilla/5.0'}
#go through proxy server if needed
proxies = {
  'http': 'http://proxy.server:8080',
  'https': 'http://proxy.server:8080',
}

base_url = "https://coder.aapc.com"
#choose a page to get all the other needed URLs
first_url = "https://coder.aapc.com/cpt-codes-range/1"

In [3]:
#get the URLs for all the pages we want to scrape
#req = requests.get(first_url, headers, proxies=proxies)
req = requests.get(first_url, headers)
source=req.text
soup = bs.BeautifulSoup(source,'lxml')
cpt_url_list = []
hcpcs_url_list = []
#the left nav bar is stored in lists classed as "link_list3"
for ul in soup.find_all("ul", class_="link_list3"):
    #pull the URLs from the hyperlinks
    for a in ul.find_all("a"):
        if(a.get('href')[0:17] == "/cpt-codes-range/"):
            cpt_url_list.append(base_url + a.get('href'))
        if(a.get('href')[0:19] == "/hcpcs-codes-range/"):
            hcpcs_url_list.append(base_url + a.get('href'))
#remove the modifiers
del cpt_url_list[-1]
del hcpcs_url_list[-1]

In [4]:
type_list = []
range_link_index_left = []
range_start = []
range_end = []
parent_cat = []
parent_range_start = []
parent_range_end = []
parent_desc = []

range_link_index_right = []
range_desc = []

for url in cpt_url_list:
    #get the source code for one page at a time
    #req = requests.get(url, headers, proxies=proxies)
    req = requests.get(url, headers)
    source=req.text
    soup = bs.BeautifulSoup(source,'lxml')
    
    #define and store what the page is displaying as a parent category for all the code ranges
    type_def = "CPT"
    parent_range_text = soup.find("meta", property="og:title")["content"]
    parent_range_text = parent_range_text[11:(len(parent_range_text)-13)].replace('\n','').replace('\r','')
    print(parent_range_text)
    parent_cat_text = parent_range_text[0:parent_range_text.rfind(" ")]
    parent_range_start_text = parent_range_text[parent_range_text.rfind(" "):parent_range_text.rfind("-")]
    parent_range_end_text = parent_range_text[(parent_range_text.rfind("-")+1):].rstrip()
    parent_desc_text = soup.find("meta", property="og:description")["content"]

    #find the left column (range) of the table that contains the ranges
    for div in soup.find_all("div", class_="bx-left"):
        for a in div.find_all('a'):
            if(a.get('href')[0:17] == "/cpt-codes-range/"):
                type_list.append(type_def)
                range_link_index_left.append(a.get('href')[18:19])
                hyphen = a.text.find("-")
                if(hyphen == -1): hyphen = 11
                #split the range into beginning and end using the hyphen
                range_start.append(a.text[0:hyphen].rstrip())
                range_end_text = a.text[(hyphen+1):11].rstrip()
                if(range_end_text == ""): range_end_text = parent_range_end_text
                range_end.append(range_end_text)
                parent_cat.append(parent_cat_text)
                parent_range_start.append(parent_range_start_text)
                parent_range_end.append(parent_range_end_text)
                parent_desc.append(parent_desc_text)

    #find the right column (description) of the table that contains the ranges
    for div in soup.find_all("div", class_="bx-right"):
        for a in div.find_all('a'):
            if(a.get('href')[0:17] == "/cpt-codes-range/"):
                range_link_index_right.append(a.get('href')[18:19])
                range_desc.append(a.text.rstrip())

for url in hcpcs_url_list:
    #get the source code for one page at a time
    #req = requests.get(url, headers, proxies=proxies)
    req = requests.get(url, headers)
    source=req.text
    soup = bs.BeautifulSoup(source,'lxml')
    
    #define and store what the page is displaying as a parent category for all the code ranges
    type_def = "HCPCS"
    parent_range_text = soup.find("meta", property="og:title")["content"]
    parent_range_text = parent_range_text[13:(len(parent_range_text)-13)].replace('\n','').replace('\r','')
    print(parent_range_text)
    parent_cat_text = parent_range_text[0:parent_range_text.rfind(" ")]
    parent_range_start_text = parent_range_text[parent_range_text.rfind(" "):parent_range_text.rfind("-")]
    parent_range_end_text = parent_range_text[(parent_range_text.rfind("-")+1):].rstrip()
    parent_desc_text = soup.find("meta", property="og:description")["content"]

    #find the left column (range) of the table that contains the ranges
    for div in soup.find_all("div", class_="bx-left"):
        for a in div.find_all('a'):
            if(a.get('href')[0:19] == "/hcpcs-codes-range/"):
                type_list.append(type_def)
                range_link_index_left.append(a.get('href')[18:19])
                hyphen = a.text.find("-")
                if(hyphen == -1): hyphen = 11
                range_start.append(a.text[0:hyphen].rstrip())
                range_end_text = a.text[(hyphen+1):11].rstrip()
                if(range_end_text == ""): range_end_text = parent_range_end_text
                range_end.append(range_end_text)
                parent_cat.append(parent_cat_text)
                parent_range_start.append(parent_range_start_text)
                parent_range_end.append(parent_range_end_text)
                parent_desc.append(parent_desc_text)
            #special case for new COVID-19 U-codes
            if(a.get('href')[22:35] == "/hcpcs-codes/"):
                type_list.append(type_def)
                range_link_index_left.append(a.get('href')[36:40])
                range_start.append(a.text.rstrip())
                range_end.append(a.text.rstrip())
                parent_cat.append(parent_cat_text)
                parent_range_start.append(parent_range_start_text)
                parent_range_end.append(parent_range_end_text)
                parent_desc.append(parent_desc_text)

    #find the right column (description) of the table that contains the ranges
    for div in soup.find_all("div", class_="bx-right"):
        for a in div.find_all('a'):
            if(a.get('href')[0:19] == "/hcpcs-codes-range/"):
                range_link_index_right.append(a.get('href')[18:19])
                range_desc.append(a.text.rstrip())
            #special case for new COVID-19 U-codes
            if(a.get('href')[22:35] == "/hcpcs-codes/"):
                range_link_index_right.append(a.get('href')[36:40])
                range_desc.append(a.text.rstrip())

Anesthesia 00100-01999
Surgery 10004-69990
Radiology Procedures 70010-79999
Pathology and Laboratory Procedures 80047-89398
Medicine Services and Procedures 90281-99756
Evaluation and Management Services 99201-99499
Category II Codes 0001F-9007F
Laboratory Analyses 0001U-0171U
Multianalyte Assay 0002M-0014M
Category III Codes 0042T-0593T
Ambulance and Other Transport Services and Supplies A0021-A0999
Medical And Surgical Supplies A4206-A8004
Administrative, Miscellaneous and Investigational A9150-A9999
Enteral and Parenteral Therapy B4034-B9999
Outpatient PPS C1713-C9899
Durable Medical Equipment E0100-E8002
Procedures / Professional Services G0008-G9987
Clinical Decision Support Mechanism G1001-G1019
Care Management Services G2021-G2058
Physical and Cognitive Status G2166-G2166
Therapy Maintenance Program in Home Health Setting G2168-G2169
Performance Measures G2167-G2167
Online Assessment by Qualified Nonphysician Healthcare Professional G2061-G2063
Comprehensive Care Management Serv

In [5]:
#if the link references match, pull the lists into a df
if(range_link_index_left == range_link_index_right):
    hcpcs_df = pd.DataFrame({
        "type": type_list,
        "range_start": range_start, 
        "range_end": range_end,
        "range_desc": range_desc,
        "parent": parent_cat,
        "parent_range_start": parent_range_start,
        "parent_range_end": parent_range_end,
        "parent_desc": parent_desc
    })   
hcpcs_df 

,type,range_start,range_end,range_desc,parent,parent_range_start,parent_range_end,parent_desc
0,CPT,00100,00222,Anesthesia for Procedures on the Head,Anesthesia,00100,01999,CPT Code range (99100–99150) for Anesthesia co...
1,CPT,00300,00352,Anesthesia for Procedures on the Neck,Anesthesia,00100,01999,CPT Code range (99100–99150) for Anesthesia co...
2,CPT,00400,00474,Anesthesia for Procedures on the Thorax (Chest...,Anesthesia,00100,01999,CPT Code range (99100–99150) for Anesthesia co...
3,CPT,00500,00580,Anesthesia for Intrathoracic Procedures,Anesthesia,00100,01999,CPT Code range (99100–99150) for Anesthesia co...
4,CPT,00600,00670,Anesthesia for Procedures on the Spine and Spi...,Anesthesia,00100,01999,CPT Code range (99100–99150) for Anesthesia co...
5,CPT,00700,00797,Anesthesia for Procedures on the Upper Abdomen,Anesthesia,00100,01999,CPT Code range (99100–99150) for Anesthesia co...
6,CPT,00800,00882,Anesthesia for Procedures on the Lower Abdomen,Anesthesia,00100,01999,CPT Code range (99100–99150) for Anesthesia co...
7,CPT,00902,00952,Anesthesia for Procedures on the Perineum,Anesthesia,00100,01999,CPT Code range (99100–99150) for Anesthesia co...
8,CPT,01112,01173,Anesthesia for Procedures on the Pelvis (Excep...,Anesthesia,00100,01999,CPT Code range (99100–99150) for Anesthesia co...
9,CPT,01200,01274,Anesthesia for Procedures on the Upper Leg (Ex...,Anesthesia,00100,01999,CPT Code range (99100–99150) for Anesthesia co...


In [6]:
#write to Excel
hcpcs_df.to_excel("aapc.xlsx")